In [1]:
"""
@author: Sadman Sakib
"""

google_colab_flag = True # Make it False if do not intend to use Google Colab and want to train in local machine!!

# For training in Google Colab
if(google_colab_flag):
  from google.colab import drive
  drive.mount('/content/drive')
  !ls
  import sys
  root_path = '/content/drive/My Drive/Practice/Stratified-k-fold-cross-validation-Image-classification-keras/' # This is the path to where in google drive the code is stored!
  sys.path.append(root_path)

# For local training
else:
  root_path = ''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


# Add all necessary libraries!

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
datasetFolderName=root_path+'DS'
MODEL_FILENAME=root_path+"model_cv.h5"
sourceFiles=[]
classLabels=['circles', 'squares', 'triangles']
X=[]
Y=[]

img_rows, img_cols =  100, 100 # input image dimensions
train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/validation/'
test_path=datasetFolderName+'/test/'

In [4]:
def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score


# First, check if test folder is empty or not, if not transfer all existing files to train.

In [5]:
transferAllClassBetweenFolders('test', 'train', 1.0)

# Now, split some part of train data into the test folders.

In [6]:
transferAllClassBetweenFolders('train', 'test', 0.20)

In [7]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
          if(folderName==classLabels[i]):
              Y.append(i)

In [8]:
# Organize file names and class labels in X and Y variables
for i in range(len(classLabels)):
  prepareNameWithLabels(classLabels[i])

In [9]:
X=np.asarray(X)
Y=np.asarray(Y)

In [10]:
# Note that, this model structure is a very basic one. To achieve better performance, you should change the model structure and hyperparameters according to your needs and data.
batch_size = 512
epoch=100
activationFunction='elu'
def getModel():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', activation=activationFunction, input_shape=(img_rows, img_cols, 3)))
    model.add(Conv2D(64, (3, 3), activation=activationFunction))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(32, (3, 3), padding='same', activation=activationFunction))
    model.add(Conv2D(32, (3, 3), activation=activationFunction))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(16, (3, 3), padding='same', activation=activationFunction))
    model.add(Conv2D(16, (3, 3), activation=activationFunction))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(64, activation=activationFunction))
    model.add(Dropout(0.1))                  
    model.add(Dense(32, activation=activationFunction))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation=activationFunction))
    model.add(Dropout(0.1))
    model.add(Dense(len(classLabels), activation='softmax')) 
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model=getModel()

# **Stratified K-Fold Cross validation**

In [11]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      zoom_range=0.20,
                      fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
        
    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)   
   
    # fit model
    history=model.fit_generator(train_generator, 
                        epochs=epoch)
    
    predictions = model.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")    
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

Results for fold 1
Found 191 images belonging to 3 classes.
Found 64 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
1/1 [==============================] - 8s 8s/step - loss: 0.6750 - accuracy: 0.3717
Epoch 2/100
1/1 [==============================] - 1s 810ms/step - loss: 0.9219 - accuracy: 0.3403
Epoch 3/100
1/1 [==============================] - 1s 798ms/step - loss: 0.7356 - accuracy: 0.3455
Epoch 4/100
1/1 [==============================] - 1s 834ms/step - loss: 0.6772 - accuracy: 0.3717
Epoch 5/100
1/1 [==============================] - 1s 827ms/step - loss: 0.6877 - accuracy: 0.3508
Epoch 6/100
1/1 [==============================] - 1s 796ms/step - loss: 0.6649 - accuracy: 0.3455
Epoch 7/100
1/1 [==============================] - 1s 798ms/step - loss: 0.6611 - accuracy: 0.3613
Epoch 8/100
1/1 [==============================] - 1s 828ms/step - loss: 0.6340 - accuracy: 0.4031
Epoch 9/100
1/1 [==============================] - 1s 795ms/step - loss: 0.6512 - accuracy: 0.3717
Epoch 10/100
1/1 [==============================] - 1s 815ms/step - loss: 0.6602 - accuracy: 0.3508
Epoch 11/100

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


1/1 [==============================] - 1s 665ms/step
***Performance on Validation data***
Accuracy  : 0.734375
Precision : 0.7534813596491228
f1Score : 0.727783436009539
[[11  2  9]
 [ 2 17  2]
 [ 2  0 19]]
Results for fold 2
Found 191 images belonging to 3 classes.
Found 64 images belonging to 3 classes.
Epoch 1/100
1/1 [==============================] - 1s 830ms/step - loss: 0.3773 - accuracy: 0.7382
Epoch 2/100
1/1 [==============================] - 1s 797ms/step - loss: 0.4026 - accuracy: 0.7330
Epoch 3/100
1/1 [==============================] - 1s 786ms/step - loss: 0.3594 - accuracy: 0.7592
Epoch 4/100
1/1 [==============================] - 1s 804ms/step - loss: 0.3749 - accuracy: 0.7749
Epoch 5/100
1/1 [==============================] - 1s 804ms/step - loss: 0.3185 - accuracy: 0.8377
Epoch 6/100
1/1 [==============================] - 1s 809ms/step - loss: 0.3191 - accuracy: 0.7801
Epoch 7/100
1/1 [==============================] - 1s 933ms/step - loss: 0.2815 - accuracy: 0.8429


# Test phase

In [12]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) 
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 45 images belonging to 3 classes.
1/1 [==============================] - 0s 452ms/step
Accuracy  : 0.8666666666666667
Precision : 0.8708333333333333
f1Score : 0.8680756395995552
[[12  0  3]
 [ 1 14  0]
 [ 2  0 13]]
